In [81]:
import ee
import folium
import geemap

In [82]:
ee.Authenticate()

True

In [83]:
ee.Initialize()

In [84]:
#import the dataset
chirps = ee.ImageCollection("UCSB-CHG/CHIRPS/DAILY")


In [85]:
# Initial date of interest (inclusive).
i_date = '2000-01-01'

## Final date of interest (exclusive).
f_date = '2024-01-01'

start_year = 2000
end_year = 2024

In [86]:
chirps_daily = chirps.select('precipitation').filterDate(i_date, f_date)

In [89]:
chirps_daily

In [90]:
#aoi
#import shapefile
ke_boundary = ee.FeatureCollection("projects/connect-gaea/assets/gadm41_KEN_0")
aoi = ke_boundary.geometry()

In [92]:
#create a rainy mask for pixels with prec>0mm
def create_rainy_mask(image, threshold = 10):  # Change the threshold value here
    rainy_mask = image.select(['precipitation']).gt(threshold)
    return image.addBands(rainy_mask.rename('rainy_mask'))

rainy_mask = chirps_daily.map(create_rainy_mask).select('rainy_mask')


## Total number of rainy days in each month for each year 



In [93]:
# Create an ee.List object with a sequence of integer numbers between a start year and an end year
years = ee.List.sequence(start_year, end_year)
# Create an ee.List object with a sequence of integer numbers between 1 and 12 (months)
months = ee.List.sequence(1, 12)

In [94]:
def yearly_monthly_sum(year):
    """Function to map the monthly_sum() to every month in the months sequences."""
    def monthly_sum(month):
        """Function to calculate the sum of rainy days for a given month in a given year."""
        w = rainy_mask.filter(ee.Filter.calendarRange(year, year, 'year')) \
                      .filter(ee.Filter.calendarRange(month, month, 'month')) \
                      .sum()
        return w.set('year', year) \
                .set('month', month) \
                .set('system:time_start', ee.Date.fromYMD(year, month, 1))
    return months.map(monthly_sum)
# Map the yearly_monthly_sum() function to every year in the years sequence to get an ImageCollection with total rainy days in each month in each year
monthly_rainy_days = ee.ImageCollection.fromImages(
    years.map(yearly_monthly_sum).flatten()
)

In [95]:


# Get the count of images in the ImageCollection
image_count = monthly_rainy_days.size().getInfo()

print(f"Number of images in 'monthly_rainy_days': {image_count}")


Number of images in 'monthly_rainy_days': 300


## total rainy days for each year

In [38]:
# Get the first image from the ImageCollection
first_image = monthly_rainy_days.first()

# Get the band names of the image
band_names = first_image.bandNames().getInfo()

print("Band names:", band_names)


Band names: ['rainy_mask']


In [98]:
# Define a function to calculate the total rainy days for each year
def yearly_total_rainy_days(year):
    """Function to calculate the total rainy days for a given year."""
    # Filter the rainy_mask ImageCollection for the current year
    yearly_data = rainy_mask.filter(ee.Filter.calendarRange(year, year, 'year'))
    
    # Sum the rainy days for the entire year
    total_rainy_days = yearly_data.sum()
    
    # Set properties for the result Image
    return total_rainy_days.set('year', year)

# List of years from 2000 to 2024
years = list(range(2000, 2025))

# Map the yearly_total_rainy_days() function to every year
# to get an ImageCollection with total rainy days in each year
yearly_rainy_days = ee.ImageCollection.fromImages(
    list(map(yearly_total_rainy_days, years))
)

# Print the yearly rainy days ImageCollection

#print("Yearly rainy days ImageCollection:", yearly_rainy_days.getInfo())


In [99]:
years

[2000,
 2001,
 2002,
 2003,
 2004,
 2005,
 2006,
 2007,
 2008,
 2009,
 2010,
 2011,
 2012,
 2013,
 2014,
 2015,
 2016,
 2017,
 2018,
 2019,
 2020,
 2021,
 2022,
 2023,
 2024]

In [105]:
yearly_rainy_days

In [106]:
# Get the first image from the ImageCollection
first_image = yearly_rainy_days.first()

# Get the band names of the image
band_names = first_image.bandNames().getInfo()

print("Band names:", band_names)


Band names: ['rainy_mask']


In [107]:
def clip_image_collection(image):
    """Function to clip each image to the area of interest shapefile."""
    return image.clip(aoi)
 

rainy_days_clipped = yearly_rainy_days.map(clip_image_collection)

In [108]:
# Check the maximum value in the image
first_Image =rainy_days_clipped.first()
max_value = first_Image.reduceRegion(
    reducer=ee.Reducer.max(),
    geometry=firstImage.geometry(),
    scale=1000  # Adjust the scale as needed
)

# Print the maximum value
print("Maximum value:", max_value.getInfo())


Maximum value: {'rainy_mask': 93}


In [117]:


# Get the number of images in the collection
num_images = rainy_days_clipped.size().getInfo()

print("Number of images in the collection:", num_images)


Number of images in the collection: 25


In [109]:
#Visualize the first image in the collection
import geemap
Map = geemap.Map()
vis_params = {
    'min': 0,
    'max': 93,
    'palette': ['blue', 'green', 'yellow', 'red']
}
firstImage =rainy_days_clipped.first()
# Add the first image to the map
Map.addLayer(firstImage, vis_params, 'First Yearly Rainy Days')

# Display the map
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [112]:
images = geemap.create_timeseries(
    yearly_rainy_days, i_date, f_date, aoi, frequency='year', reducer='median'
)
images

## average rainy days for each month across all years

In [113]:
def monthly_mean(month):
    """Function to calculate the average rainy days for a given month across all years."""
    # Round to the nearest integer.
    w = monthly_rainy_days.filter(ee.Filter.eq('month', month)).mean().round()
    return w.set('month', month) \
            .set('system:time_start', ee.Date.fromYMD(1, month, 1))
 
# Map the monthly_mean() function to every month in the months sequence and make an ImageCollection from the obtained images
mean_monthly_rainy_days = ee.ImageCollection.fromImages(
    months.map(monthly_mean).flatten()
)

## clip to aoi

In [114]:
def clip_image_collection(image):
    """Function to clip each image to the area of interest shapefile."""
    return image.clip(aoi)
 
# Map the clip_image_collection() function to the mean_monthly_rainy_days collection 
mean_monthly_rainy_days_clipped = mean_monthly_rainy_days.map(clip_image_collection)

## Lets plot

In [115]:
Map = geemap.Map()
Map.centerObject(aoi, 4)
precipitation_vis = {
    'min': 1,
    'max': 17,
    'palette': ['001137', '0aab1e', 'e7eb05', 'ff4a2d', 'e90000'],
}
months = mean_monthly_rainy_days_clipped.aggregate_array('system:index').getInfo()
months

['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11']

In [116]:
Map.ts_inspector(
    left_ts=mean_monthly_rainy_days_clipped,
    right_ts=mean_monthly_rainy_days_clipped,
    left_names=months,
    right_names=months,
    left_vis=precipitation_vis,
    right_vis=precipitation_vis,
    width='80px',
)
Map

Map(center=[0.5524335862141333, 37.84581141226501], controls=(WidgetControl(options=['position', 'transparent_…